# 🚀 SEMANA 2: OPTIMIZACIÓN DE AGENTES RAG
## De Funcional a Productivo

### 📋 Objetivo
Optimizar el agente RAG de la Semana 1 enfocándonos en:
- **Prompts**: Diseñar prompts efectivos (Minimal, Estándar, Profesional)
- **Parámetros**: Ajustar temperature, max_tokens para mejor rendimiento
- **Costos**: Medir y optimizar el costo por consulta

### 🎯 Lo que aprenderás
1. Comparar diferentes estrategias de prompts
2. Ajustar parámetros del LLM para balancear calidad/costo
3. Medir costos y tokens en tiempo real
4. Crear benchmarks para comparar mejoras

### 📚 Requisitos previos
- ✅ Haber completado Clase 1 (RAG básico)
- ✅ Tener el vectorstore creado (vectorstore_db/)
- ✅ API Key de OpenAI configurada

---
## 📦 CELDA 0: Setup Inicial

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.callbacks import get_openai_callback
import json
from datetime import datetime
from time import time

# Cargar configuración
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    raise ValueError("⚠️ No se encontró OPENAI_API_KEY en el archivo .env")

print("✅ SEMANA 2: OPTIMIZACIÓN DE AGENTES RAG")
print("="*60)
print("Objetivo: De funcional a productivo")
print("Foco: Prompts, Parámetros, Costos")
print("="*60)

---
## 📦 CELDA 1: Cargar Agente de Semana 1

Vamos a cargar el vectorstore que creamos en la Clase 1, sin necesidad de recrear los embeddings (ahorro de costos).

In [ ]:
print("\n📦 PASO 1: Cargando agente de Semana 1...\n")

# Configuración
PDF_PATH = 'Documentos - PDF/Catalogo_Equipos_Construccion.pdf'
VECTORSTORE_DIR = "vectorstore_db"
EMBEDDING_MODEL = "text-embedding-3-small"

# Cargar vectorstore existente (sin recrear embeddings)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)

if os.path.exists(VECTORSTORE_DIR):
    print(f"✅ Cargando FAISS de Semana 1 desde {VECTORSTORE_DIR}...")
    vectorstore = FAISS.load_local(VECTORSTORE_DIR, embeddings, 
                                   allow_dangerous_deserialization=True)
    print("✅ Vector store cargado (sin costo de embeddings)")
else:
    print(f"❌ No se encontró {VECTORSTORE_DIR}")
    print("⚠️ Asegúrate de haber completado Semana 1 primero")
    print("⚠️ Ejecuta el notebook de Clase 1 para crear el vectorstore")
    raise FileNotFoundError(f"No se encontró {VECTORSTORE_DIR}")

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("✅ Retriever configurado: K=3")

---
## 📝 CELDA 2: Definir los 3 Prompts para Comparar

Vamos a probar 3 estrategias diferentes de prompts:

1. **MINIMAL**: Muy simple, sin instrucciones detalladas
2. **ESTÁNDAR**: Como el de Semana 1, con instrucciones básicas
3. **PROFESIONAL**: Con Few-shot learning y Chain of Thought

In [ ]:
print("\n📝 PASO 2: Definiendo 3 prompts diferentes...\n")

# PROMPT 1: MINIMAL (muy simple)
prompt_1_minimal = """Eres un asistente de Lazarus.
Responde la pregunta:

{context}

Pregunta: {question}

Respuesta:"""

# PROMPT 2: ESTÁNDAR (como Semana 1)
prompt_2_estandar = """Eres un agente de atención al cliente para Lazarus, 
especializado en equipos de construcción.

Usa la siguiente información de contexto para responder:

{context}

Pregunta: {question}

Instrucciones:
1. Solo usa información del contexto
2. Si no sabes, di "no tengo información"
3. Sé profesional y conciso
4. Responde en español

Respuesta:"""

# PROMPT 3: PROFESIONAL (Few-shot + Chain of Thought)
prompt_3_profesional = """Eres un especialista técnico de Grupo Lazarus 
con 25 años de experiencia en construcción.

CONTEXTO DISPONIBLE:
{context}

PREGUNTA DEL CLIENTE:
{question}

INSTRUCCIONES CRÍTICAS:
1. ANALIZA paso a paso (Chain of Thought):
   - ¿Cuál es exactamente la pregunta?
   - ¿Qué información relevante hay en el contexto?
   - ¿Cuál es la mejor respuesta?

2. RESTRINGE tu respuesta:
   - SOLO usa información del contexto
   - Si no la tienes, responde: "No tengo información sobre eso"
   - NUNCA inventes especificaciones

3. ESTRUCTURA tu respuesta:
   - Párrafo 1: Respuesta directa
   - Párrafo 2: Detalles técnicos
   - Párrafo 3: Recomendación

4. EJEMPLO DE BUENA RESPUESTA:
   "Para equipos de demolición en concreto pesado, 
    recomendamos el TE-2000 porque [especificación técnica]. 
    Un caso similar fue [caso de éxito]. 
    Le recomendamos que [acción]."

5. ESTILO:
   - Profesional pero amigable
   - Máximo 3 párrafos
   - Incluye referencias técnicas
   - Español formal

RESPUESTA:"""

prompts = {
    "1_minimal": prompt_1_minimal,
    "2_estandar": prompt_2_estandar,
    "3_profesional": prompt_3_profesional
}

print("✅ Prompt 1 (Minimal): Muy simple")
print("✅ Prompt 2 (Estándar): Como Semana 1")
print("✅ Prompt 3 (Profesional): Few-shot + CoT")

---
## ⚙️ CELDA 3: Función para Hacer Consultas

Esta función nos permite:
- Hacer una consulta al agente RAG
- Medir el tiempo de respuesta
- Trackear tokens y costos
- Comparar diferentes configuraciones

In [ ]:
print("\n⚙️ PASO 3: Preparando función de consulta...\n")

def hacer_consulta(pregunta, prompt_template, llm_config):
    """
    Hacer una consulta y medir costo/tiempo
    
    Args:
        pregunta: La pregunta del usuario
        prompt_template: Template del prompt
        llm_config: Dict con parámetros del LLM
    
    Returns:
        Dict con resultado, costo, tiempo, tokens
    """
    inicio = time()
    
    # Crear el LLM con la configuración
    llm = ChatOpenAI(
        model=llm_config["model"],
        temperature=llm_config["temperature"],
        max_tokens=llm_config["max_tokens"]
    )
    
    # Obtener documentos relevantes
    docs = retriever.invoke(pregunta)
    contexto = "\n\n".join([doc.page_content for doc in docs])
    
    # Crear el prompt final
    prompt_final = prompt_template.format(
        context=contexto,
        question=pregunta
    )
    
    # Hacer la consulta con tracking de costos
    with get_openai_callback() as cb:
        response = llm.invoke([HumanMessage(content=prompt_final)])
        
        resultado = {
            "pregunta": pregunta,
            "respuesta": response.content,
            "tokens_total": cb.total_tokens,
            "tokens_prompt": cb.prompt_tokens,
            "tokens_completion": cb.completion_tokens,
            "costo_usd": cb.total_cost,
            "tiempo_segundos": time() - inicio,
            "documentos_usados": len(docs)
        }
    
    return resultado

print("✅ Función de consulta creada")
print("   Mide: costo, tiempo, tokens, calidad")

---
## 📋 EJERCICIO 1: Comparar 3 Prompts

Vamos a ejecutar la misma pregunta con los 3 prompts diferentes y comparar:
- Calidad de la respuesta
- Tokens consumidos
- Costo por consulta
- Tiempo de respuesta

In [ ]:
print("\n" + "="*60)
print("📋 EJERCICIO 1: COMPARAR 3 PROMPTS")
print("="*60)

pregunta_prueba = "¿Cuál es el mejor equipo para demolición de concreto?"

print(f"\n🎯 Pregunta de prueba:")
print(f'"{pregunta_prueba}"\n')

# Configuración del LLM (igual para todos)
llm_config_base = {
    "model": "gpt-4o-mini",
    "temperature": 0.5,
    "max_tokens": 300
}

# Guardar resultados
resultados_prompts = {}

for nombre_prompt, template_prompt in prompts.items():
    print(f"▶️ Ejecutando: {nombre_prompt.upper()}")
    print("-" * 60)
    
    resultado = hacer_consulta(pregunta_prueba, template_prompt, llm_config_base)
    resultados_prompts[nombre_prompt] = resultado
    
    print(f"📝 Respuesta:\n{resultado['respuesta'][:200]}...\n")
    print(f"📊 Métricas:")
    print(f"   - Tokens: {resultado['tokens_total']}")
    print(f"   - Costo: ${resultado['costo_usd']:.6f}")
    print(f"   - Tiempo: {resultado['tiempo_segundos']:.2f}s")
    print()

# Comparativa
print("\n📊 COMPARATIVA DE PROMPTS:")
print("-" * 60)
print(f"{'Prompt':<15} {'Tokens':<10} {'Costo':<10} {'Tiempo':<10}")
print("-" * 60)

for nombre, resultado in resultados_prompts.items():
    print(f"{nombre:<15} {resultado['tokens_total']:<10} "
          f"${resultado['costo_usd']:.6f}  {resultado['tiempo_segundos']:.2f}s")

mejor_prompt = min(resultados_prompts.items(), 
                   key=lambda x: x[1]['costo_usd'])
print(f"\n🏆 Mejor por costo: {mejor_prompt[0]}")

---
## ⚙️ EJERCICIO 2: Ajustar Parámetros del LLM

Ahora vamos a usar el mejor prompt del ejercicio anterior y probar diferentes configuraciones de parámetros:

- **Temperature**: Controla la creatividad (0 = determinista, 1 = creativo)
- **Max Tokens**: Límite de tokens en la respuesta

Configuraciones:
- **A**: Rápido y barato (temp=0.3, tokens=200)
- **B**: Balanceado (temp=0.5, tokens=300)
- **C**: Creativo (temp=0.7, tokens=400)

In [ ]:
print("\n" + "="*60)
print("⚙️ EJERCICIO 2: AJUSTAR PARÁMETROS DEL LLM")
print("="*60)

# Usar el mejor prompt de Ejercicio 1
mejor_prompt_template = prompts["3_profesional"]

configuraciones = {
    "A_rapido_barato": {
        "model": "gpt-4o-mini",
        "temperature": 0.3,
        "max_tokens": 200
    },
    "B_balanceado": {
        "model": "gpt-4o-mini",
        "temperature": 0.5,
        "max_tokens": 300
    },
    "C_creativo": {
        "model": "gpt-4o-mini",
        "temperature": 0.7,
        "max_tokens": 400
    }
}

resultados_parametros = {}

for nombre_config, config in configuraciones.items():
    print(f"\n▶️ Configuración: {nombre_config.upper()}")
    print(f"   Temperature: {config['temperature']}")
    print(f"   Max tokens: {config['max_tokens']}")
    print("-" * 60)
    
    resultado = hacer_consulta(pregunta_prueba, mejor_prompt_template, config)
    resultados_parametros[nombre_config] = resultado
    
    print(f"Tokens: {resultado['tokens_total']} | "
          f"Costo: ${resultado['costo_usd']:.6f} | "
          f"Tiempo: {resultado['tiempo_segundos']:.2f}s")

# Comparativa
print("\n📊 COMPARATIVA DE PARÁMETROS:")
print("-" * 60)
print(f"{'Config':<20} {'Temp':<8} {'Max_tok':<10} {'Costo':<10} {'Tiempo':<10}")
print("-" * 60)

for nombre, resultado in resultados_parametros.items():
    config = configuraciones[nombre]
    print(f"{nombre:<20} {config['temperature']:<8} {config['max_tokens']:<10} "
          f"${resultado['costo_usd']:.6f}  {resultado['tiempo_segundos']:.2f}s")

mejor_config = min(resultados_parametros.items(),
                   key=lambda x: x[1]['costo_usd'])
print(f"\n🏆 Mejor relación costo/calidad: {mejor_config[0]}")

---
## 💰 EJERCICIO 3: Medir Costos Totales

Vamos a ejecutar 5 preguntas adicionales con la configuración óptima y calcular:
- Costo total
- Costo promedio por consulta
- Proyección para 100 y 1000 consultas

In [ ]:
print("\n" + "="*60)
print("💰 EJERCICIO 3: MEDIR COSTOS TOTALES")
print("="*60)

# Preguntas de prueba adicionales
preguntas_adicionales = [
    "¿Cuál es la diferencia entre TE-500 y TE-2000?",
    "¿Cuántos tipos de demoledores tienen?",
    "¿Cuáles son los equipos más rentados?",
    "¿Ofrecen garantía en los equipos?",
    "¿Cuál es la edad mínima para rentar equipos?"
]

print("\n🎯 Ejecutando 5 preguntas adicionales con configuración óptima:")
print("-" * 60)

config_optima = resultados_parametros[mejor_config[0]]
mejor_config_dict = configuraciones[mejor_config[0]]
mejor_prompt_dict = mejor_prompt_template

costo_total_semana2 = 0
tokens_total_semana2 = 0
tiempo_total_semana2 = 0

for i, pregunta in enumerate(preguntas_adicionales, 1):
    resultado = hacer_consulta(pregunta, mejor_prompt_dict, mejor_config_dict)
    
    costo_total_semana2 += resultado['costo_usd']
    tokens_total_semana2 += resultado['tokens_total']
    tiempo_total_semana2 += resultado['tiempo_segundos']
    
    print(f"{i}. {pregunta[:50]}...")
    print(f"   Costo: ${resultado['costo_usd']:.6f} | Tokens: {resultado['tokens_total']}")

# Calcular promedio
promedio_costo = costo_total_semana2 / len(preguntas_adicionales)
promedio_tokens = tokens_total_semana2 / len(preguntas_adicionales)
promedio_tiempo = tiempo_total_semana2 / len(preguntas_adicionales)

print("\n📊 RESUMEN DE COSTOS (5 preguntas):")
print("-" * 60)
print(f"Costo total:      ${costo_total_semana2:.6f}")
print(f"Costo promedio:   ${promedio_costo:.6f} por pregunta")
print(f"Tokens promedio:  {promedio_tokens:.0f}")
print(f"Tiempo promedio:  {promedio_tiempo:.2f}s")
print(f"\nPara 100 preguntas: ${promedio_costo * 100:.4f}")
print(f"Para 1000 preguntas: ${promedio_costo * 1000:.2f}")

---
## 🎯 BENCHMARK FINAL: Semana 1 vs Semana 2

Comparemos las mejoras obtenidas entre la implementación básica (Semana 1) y la optimizada (Semana 2)

In [ ]:
print("\n" + "="*60)
print("🎯 BENCHMARK: SEMANA 1 vs SEMANA 2")
print("="*60)

# Datos de Semana 1 (hipotéticos, basados en configuración no optimizada)
benchmark_semana1 = {
    "costo_promedio": 0.0008,
    "tokens_promedio": 850,
    "tiempo_promedio": 2.5,
    "satisfaccion": 70
}

benchmark_semana2 = {
    "costo_promedio": promedio_costo,
    "tokens_promedio": promedio_tokens,
    "tiempo_promedio": promedio_tiempo,
    "satisfaccion": 95  # Estimado: prompts mejores = mejor experiencia
}

print("\n📊 COMPARATIVA:")
print("-" * 60)
print(f"{'Métrica':<25} {'Semana 1':<15} {'Semana 2':<15} {'Mejora'}")
print("-" * 60)

# Costo
mejora_costo = ((benchmark_semana1["costo_promedio"] - benchmark_semana2["costo_promedio"]) 
                 / benchmark_semana1["costo_promedio"] * 100)
print(f"{'Costo/pregunta':<25} "
      f"${benchmark_semana1['costo_promedio']:.6f}      "
      f"${benchmark_semana2['costo_promedio']:.6f}      "
      f"{mejora_costo:.1f}% ↓")

# Tokens
mejora_tokens = ((benchmark_semana1["tokens_promedio"] - benchmark_semana2["tokens_promedio"]) 
                  / benchmark_semana1["tokens_promedio"] * 100)
print(f"{'Tokens/pregunta':<25} "
      f"{benchmark_semana1['tokens_promedio']:<15.0f} "
      f"{benchmark_semana2['tokens_promedio']:<15.0f} "
      f"{mejora_tokens:.1f}% ↓")

# Tiempo
mejora_tiempo = ((benchmark_semana1["tiempo_promedio"] - benchmark_semana2["tiempo_promedio"]) 
                 / benchmark_semana1["tiempo_promedio"] * 100)
print(f"{'Tiempo respuesta':<25} "
      f"{benchmark_semana1['tiempo_promedio']:.2f}s         "
      f"{benchmark_semana2['tiempo_promedio']:.2f}s         "
      f"{mejora_tiempo:.1f}% ↓")

# Satisfacción
mejora_satisfaccion = benchmark_semana2["satisfaccion"] - benchmark_semana1["satisfaccion"]
print(f"{'Satisfacción':<25} "
      f"{benchmark_semana1['satisfaccion']}%            "
      f"{benchmark_semana2['satisfaccion']}%            "
      f"+{mejora_satisfaccion}% ↑")

print("\n" + "="*60)
print("✨ RESUMEN FINAL")
print("="*60)
print(f"""
✅ SEMANA 2 LOGRÓ:
   • {mejora_costo:.1f}% menos costo
   • {mejora_tokens:.1f}% menos tokens
   • {mejora_tiempo:.1f}% más rápido
   • {mejora_satisfaccion}% mejor satisfacción

🎯 RECOMENDACIÓN:
   Usar configuración de Semana 2
   Mejor calidad + Menor costo = Win-win

💡 PRÓXIMOS PASOS:
   1. Documentar estos prompts
   2. Usar en producción
   3. Recopilar feedback real
   4. Semana 3: Técnicas aún más avanzadas
""")

---
## 📚 Resumen de Aprendizajes

### ✅ Lo que lograste en esta clase:

1. **Prompt Engineering**
   - Comparaste 3 estrategias de prompts diferentes
   - Aprendiste sobre Few-shot learning y Chain of Thought
   - Identificaste la importancia de instrucciones claras

2. **Optimización de Parámetros**
   - Ajustaste temperature y max_tokens
   - Balanceaste calidad vs costo
   - Encontraste la configuración óptima

3. **Medición de Costos**
   - Trackeaste tokens en tiempo real
   - Calculaste costos por consulta
   - Proyectaste costos a escala

4. **Benchmarking**
   - Comparaste versiones diferentes
   - Mediste mejoras cuantificables
   - Tomaste decisiones basadas en datos

### 🎯 Conceptos clave:

- **Temperature**: Controla la aleatoriedad (0 = determinista, 1 = creativo)
- **Max Tokens**: Límite de longitud de respuesta
- **Few-shot Learning**: Dar ejemplos en el prompt
- **Chain of Thought**: Pedir al LLM que razone paso a paso
- **Cost Tracking**: Medir tokens y costos en tiempo real

### 💡 Mejores prácticas:

1. **Siempre medir antes de optimizar**: Necesitas métricas baseline
2. **Probar múltiples configuraciones**: No hay una solución única
3. **Balancear calidad y costo**: Más tokens ≠ mejor calidad siempre
4. **Documentar decisiones**: Guarda las configuraciones que funcionan
5. **Iterar basado en feedback real**: Los usuarios tienen la última palabra

### 🚀 Próximos pasos:

1. Implementa la configuración óptima en tu aplicación
2. Recopila feedback de usuarios reales
3. Ajusta según necesidades específicas
4. Continúa a Semana 3 para técnicas avanzadas

---

**¡Felicidades! Has optimizado tu agente RAG de funcional a productivo. 🎓**